In [1]:
pip install --upgrade typing_extensions


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time
import random

def get_indeed_jobs(search_query, pages=1):
    driver = webdriver.Chrome()
    driver.get("https://www.indeed.com/")
    
    try:
        # Wait for the 'what' field and enter the search query
        what_field = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "text-input-what")))
        what_field.clear()
        what_field.send_keys(search_query)
        what_field.send_keys(Keys.RETURN)
        
        # Click on the date filter button with the provided XPath
        try:
            date_filter_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, "/html/body/main/div/div[2]/div/div[5]/div/div[1]/div[4]/div/div/div[1]/span[2]/b"))
            )
            date_filter_button.click()
            time.sleep(1)  # Allow page update
        except:
            print("Date filter button not found or clickable.")
        
        job_data = []
        
        for _ in range(pages):
            # Scroll to the bottom to load all job cards
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(2)  # Pause to load content

            # Locate job cards with a broader selector
            job_cards = WebDriverWait(driver, 15).until(
                EC.presence_of_all_elements_located((By.XPATH, '//div[contains(@class, "job_seen_beacon")]'))
            )
            
            print(f"Found {len(job_cards)} job cards on the page")

            for i in range(1, len(job_cards) + 1):
                try:
                    # Extract the name of the job
                    job_title = driver.find_element(By.XPATH, f'/html/body/main/div/div[2]/div/div[5]/div/div[1]/div[5]/div/ul/li[{i}]/div/div/div/div/div/div/table/tbody/tr/td[1]/div[1]/h2/a').text
                except:
                    job_title = "N/A"
                
                try:
                    # Extract the name of the employer
                    employer = driver.find_element(By.XPATH, f'/html/body/main/div/div[2]/div/div[5]/div/div[1]/div[5]/div/ul/li[{i}]/div/div/div/div/div/div/table/tbody/tr/td[1]/div[2]/div/div[1]').text
                except:
                    employer = "N/A"
                
                try:
                    # Extract the name of the city
                    city = driver.find_element(By.XPATH, f'/html/body/main/div/div[2]/div/div[5]/div/div[1]/div[5]/div/ul/li[{i}]/div/div/div/div/div/div/table/tbody/tr/td[1]/div[2]/div/div[2]').text
                except:
                    city = "N/A"
                
                try:
                    # Extract the type of employment
                    employment_type = driver.find_element(By.XPATH, f'/html/body/main/div/div[2]/div/div[5]/div/div[1]/div[5]/div/ul/li[{i}]/div/div/div/div/div/div/table/tbody/tr/td[1]/div[3]/ul/li/div').text
                except:
                    employment_type = "N/A"

                # Append job details to list
                job_data.append({
                    "Job Title": job_title,
                    "Employer": employer,
                    "City": city,
                    "Employment Type": employment_type
                })

            # Attempt to go to the next page if available
            try:
                next_button = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, '//a[@aria-label="Next"]')))
                next_button.click()
                time.sleep(random.uniform(1, 3))  # Random sleep to mimic human behavior
            except:
                print("No more pages available.")
                break  # No more pages
        
    finally:
        driver.quit()
    
    # Convert list to DataFrame
    jobs_df = pd.DataFrame(job_data)
    return jobs_df

# Usage
search_query = "Data Scientist"
df_jobs = get_indeed_jobs(search_query, pages=2)

print(df_jobs)
df_jobs.to_csv("indeed_job_listings.csv", index=False)


Found 15 job cards on the page
No more pages available.
                                          Job Title  \
0   Software Engineer for AI Training Data (Python)   
1                               Junior Statistician   
2                                      Data Analyst   
3                 Data Scientist, Data valorization   
4                                      Data Analyst   
5                                               N/A   
6                              Data Science Analyst   
7                                    Data Scientist   
8             Artificial Intelligence Scientist, II   
9             Data scientist (intermédiaire/senior)   
10               Oliver Wyman - Lead Data Scientist   
11                                              N/A   
12                        Senior R&D Data Scientist   
13         Senior Data Scientist, Data valorization   
14                            Senior Data Scientist   

                              Employer                         

In [3]:
df_jobs.head()

,Job Title,Employer,City,Employment Type
0,Software Engineer for AI Training Data (Python),G2i Inc,"Remote in Montréal, QC",Contract
1,Junior Statistician,Confidential,"Montréal, QC","$45,000–$50,000 a year"
2,Data Analyst,NRG PERSONNEL,"Montréal, QC H1E 6M3","$75,000–$85,000 a year"
3,"Data Scientist, Data valorization",Desjardins,"Hybrid work in Montréal, QC",Full-time
4,Data Analyst,Shivansh Outsourcing,"Remote in Montréal, QC","$65,000–$70,000 a year"
